In [1]:
import warnings
from xml.etree.ElementInclude import include
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import pandas as pd
from functools import reduce
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from statistics import mean
import numpy as np
import statsmodels.api as sm


import matplotlib
plt.style.use('ggplot')

matplotlib.rcParams['figure.figsize'] = (12,8)

pd.options.mode.chained_assignment = None

import seaborn as sns


In [2]:
df = pd.read_csv("../data/telcom.csv")

In [6]:
experience_df = pd.DataFrame({'customer':df['MSISDN/Number'],
                      'sessions_frequency':df['Bearer Id'],
                      'duration':df['Dur. (ms)'],
                        'handset':df['Handset Type']})
experience_df['rtt'] = df['Avg RTT DL (ms)']+df['Avg RTT UL (ms)']
experience_df['tp'] = df['Avg Bearer TP DL (kbps)']+df['Avg Bearer TP UL (kbps)']
experience_df['tcp'] = df['TCP DL Retrans. Vol (Bytes)']+df['TCP UL Retrans. Vol (Bytes)']
experience_df.head(4)

,customer,sessions_frequency,duration,handset,rtt,tp,tcp
0,3.366496e+10,1.311450e+19,1823652.0,Samsung Galaxy A5 Sm-A520F,47.0,67.0,NaN
1,3.368185e+10,1.311450e+19,1365104.0,Samsung Galaxy J5 (Sm-J530),70.0,42.0,NaN
2,3.376063e+10,1.311450e+19,1361762.0,Samsung Galaxy A8 (2018),NaN,15.0,NaN
3,3.375034e+10,1.311450e+19,1321509.0,undefined,NaN,88.0,NaN


In [8]:
def generate_pipeline(type_="numeric",x=1):
    pipeline = None
    if type_ == "numeric":
        pipeline = Pipeline(steps=[
            ('impute', SimpleImputer(strategy='mean')),
            ('scale', MinMaxScaler())
        ])
    elif type_ == "categorical":
        pipeline = Pipeline(steps=[
        ('impute', SimpleImputer(strategy='most_frequent')),
        ('one-hot', OneHotEncoder(handle_unknown='ignore', sparse=False))
        ])
    else:
        pipeline = np.zeros(x)
    return pipeline

In [9]:
numeric_pipeline = generate_pipeline("numeric")
assert isinstance(numeric_pipeline,Pipeline)

In [10]:
categorical_pipeline = generate_pipeline("categorical")
assert isinstance(numeric_pipeline,Pipeline)

In [11]:
def store_features(df,type_,value):
    features = [None]
    if type_ == "numeric":
        features = df.select_dtypes(include=value).columns.tolist()
    elif type_ == "categorical":
        features = df.select_dtypes(exclude=value).columns.tolist()
    return features


In [19]:
categorical_features = store_features(experience_df,"categorical","number")

In [20]:
numerical_features = store_features(experience_df,"numeric","number")

In [21]:
def handle_missing_values_numeric(df,features):
    """
    this algorithm does the following
    - remove columns with x percentage of missing values
    - fill the missing values with the mean
    returns:
        - df
        - percentage of missing values
    """
    missing_percentage = round((df.isnull().sum().sum()/\
            reduce(lambda x, y: x*y, df.shape))*100,2)
    for key in features:
        df[key] = df[key].fillna(df[key].mean())
    return missing_percentage, df

In [22]:
handle_missing_values_numeric(experience_df,numerical_features)

(12.38,
             customer  sessions_frequency      duration  \
 0       3.366496e+10        1.311450e+19  1.823652e+06   
 1       3.368185e+10        1.311450e+19  1.365104e+06   
 2       3.376063e+10        1.311450e+19  1.361762e+06   
 3       3.375034e+10        1.311450e+19  1.321509e+06   
 4       3.369980e+10        1.311450e+19  1.089009e+06   
 ...              ...                 ...           ...   
 149996  3.365069e+10        7.277830e+18  8.123000e+04   
 149997  3.366345e+10        7.349880e+18  9.797000e+04   
 149998  3.362189e+10        1.311450e+19  9.824900e+04   
 149999  3.361962e+10        1.311450e+19  9.791000e+04   
 150000  4.188282e+10        1.013887e+19  1.046086e+05   
 
                             handset         rtt            tp           tcp  
 0        Samsung Galaxy A5 Sm-A520F   47.000000     67.000000  2.571697e+07  
 1       Samsung Galaxy J5 (Sm-J530)   70.000000     42.000000  2.571697e+07  
 2          Samsung Galaxy A8 (2018)  127.024

In [23]:
def handle_missing_values_categorical(df,features):
    """
    this algorithm does the following
    - remove columns with x percentage of missing values
    - fill the missing values with the mode
    returns:
        - df
        - percentage of missing values
    """
    missing_percentage = round((df.isnull().sum().sum()/\
            reduce(lambda x, y: x*y, df.shape))*100,2)
    for key in features:
        df[key] = df[key].fillna(df[key].mode()[0])
    return missing_percentage, df

In [24]:
handle_missing_values_categorical(experience_df,categorical_features)

(0.05,
             customer  sessions_frequency      duration  \
 0       3.366496e+10        1.311450e+19  1.823652e+06   
 1       3.368185e+10        1.311450e+19  1.365104e+06   
 2       3.376063e+10        1.311450e+19  1.361762e+06   
 3       3.375034e+10        1.311450e+19  1.321509e+06   
 4       3.369980e+10        1.311450e+19  1.089009e+06   
 ...              ...                 ...           ...   
 149996  3.365069e+10        7.277830e+18  8.123000e+04   
 149997  3.366345e+10        7.349880e+18  9.797000e+04   
 149998  3.362189e+10        1.311450e+19  9.824900e+04   
 149999  3.361962e+10        1.311450e+19  9.791000e+04   
 150000  4.188282e+10        1.013887e+19  1.046086e+05   
 
                             handset         rtt            tp           tcp  
 0        Samsung Galaxy A5 Sm-A520F   47.000000     67.000000  2.571697e+07  
 1       Samsung Galaxy J5 (Sm-J530)   70.000000     42.000000  2.571697e+07  
 2          Samsung Galaxy A8 (2018)  127.0248

Task 3. 1 - Aggregate, per customer, the following information (treat missing & outliers by replacing by the mean or the mode of the corresponding variable):
Average TCP retransmission
Average RTT
Handset type
Average throughput


In [33]:
def aggregation_cols(df,col_1,col_2,ty="numerical",trim=False):
    if ty == "numerical":
        grouped = df.groupby(col_1).agg({col_2: [min, max, mean]}) 
        grouped.columns = ["_".join(x) for x in grouped.columns.ravel()]
    else:
        grouped = df.groupby(col_1).agg({col_2: [mode]}) 
        grouped.columns = ["_".join(x) for x in grouped.columns.ravel()]
    if trim:
        return grouped.describe()
    return grouped

In [28]:
tcp_aggregation = aggregation_cols(experience_df,'customer','tcp')
top_customers_tcp = tcp_aggregation.sort_values(by='tcp_max', ascending=False)
top_customers_tcp.head(10)

,tcp_min,tcp_max,tcp_mean
customer,,,
3.366045e+10,1.861868e+07,4.344116e+09,2.181367e+09
3.369858e+10,4.301477e+09,4.301477e+09,4.301477e+09
3.376211e+10,3.268309e+08,4.294432e+09,2.310631e+09
3.365872e+10,1.109669e+07,4.291465e+09,1.438251e+09
3.365871e+10,4.288121e+09,4.288121e+09,4.288121e+09
3.366469e+10,6.390463e+06,4.275492e+09,9.130163e+08
3.366491e+10,4.268647e+09,4.268647e+09,4.268647e+09
3.376233e+10,1.959403e+06,4.261616e+09,2.131788e+09
3.365828e+10,3.435268e+06,4.260174e+09,9.562750e+08


In [27]:
rtt_aggregation = aggregation_cols(experience_df,'customer','rtt')
top_customers_rtt = rtt_aggregation.sort_values(by='rtt_max', ascending=False)
top_customers_rtt.head(10)

,rtt_min,rtt_max,rtt_mean
customer,,,
3.366232e+10,96924.0,96924.0,96924.000000
3.366087e+10,29.0,64641.0,32335.000000
4.188282e+10,19.0,55812.0,309.155397
3.368369e+10,54848.0,54848.0,54848.000000
3.376094e+10,27278.0,27278.0,27278.000000
3.360679e+10,26300.0,26300.0,26300.000000
3.369855e+10,20099.0,25922.0,23010.500000
3.367182e+10,25715.0,25715.0,25715.000000
3.378187e+10,25388.0,25388.0,25388.000000


In [34]:
tp_aggregation = aggregation_cols(experience_df,'customer','tp')
top_customers_tp = tp_aggregation.sort_values(by='tp_max', ascending=False)
top_customers_tp.head(10)

,tp_min,tp_max,tp_mean
customer,,,
3.366117e+10,37641.0,382262.0,156073.333333
3.367251e+10,2.0,313244.0,128630.500000
3.369817e+10,2.0,304299.0,103132.833333
3.366662e+10,122.0,300546.0,150334.000000
3.365991e+10,283931.0,283931.0,283931.000000
3.366566e+10,249209.0,281144.0,265176.500000
3.365169e+10,86.0,277152.0,138619.000000
3.366856e+10,109.0,276205.0,138157.000000
3.366549e+10,101.0,274052.0,78734.166667


In [ ]:
tp_aggregation = aggregation_cols(experience_df,'customer','tp')
top_customers_tp = tp_aggregation.sort_values(by='tp_max', ascending=False)
top_customers_tp.head(10)